In [12]:
#Question 1 : Extract the Balance Sheet, Income Statement, and Cash Flow tables from the document (if they exist).
'''API's of Multimodal LLMs aren't freely available anymore. Using the API Keys of several LLMs like OpenAI, Gemini, Claude, Grok requires
   high prices to extensively use the API for the task. So, I, finally took the task in hand and used OCR for text recognition
   from the PDF file and developed python scripts to extract the Balance Sheet, Income Statement and Cash Flow tables from the document. '''

# Install necessary packages
!apt-get install -y poppler-utils tesseract-ocr
!pip install pdf2image pytesseract opencv-python pandas markdownify tabulate

import os
import cv2
import pytesseract
import numpy as np
import pandas as pd
from pdf2image import convert_from_path
from markdownify import markdownify as md
from tabulate import tabulate

# Define PDF path
PDF_PATH = "/content/vodafone_annual_report_reduced.pdf"

# Convert PDF pages to images
images = convert_from_path(PDF_PATH, dpi=300)

# Function to preprocess image for OCR
def preprocess_image(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

# Extract tables using OCR
def extract_text_from_image(image):
    processed_img = preprocess_image(image)
    extracted_text = pytesseract.image_to_string(processed_img, config="--psm 6")  # OCR
    return extracted_text

# Extract text from all images
extracted_text = [extract_text_from_image(img) for img in images]
full_text = "\n".join(extracted_text)

# Save extracted text for debugging
with open("extracted_text.txt", "w") as f:
    f.write(full_text)

# Vocabulary List for Standardization
vocabulary_mapping = {
    "Cash and Equivalents": "Cash",
    "Trade Receivables": "Accts Rec-Trade (Trade Debtors)",
    "Inventory": "Inventory (Stock)",
    "Tax Receivable": "Tax Receivable",
    "Other Current Assets": "Other Current Assets / Other Debtors",
    "Current Related Party Assets": "Current Related Party Assets",
    "Prepayments and Accrued Income": "Prepayments and accrued income",
    "Land & Buildings": "Land & Buildings",
    "Plant & Equipment": "Plant & Machinery",
    "Non Current Receivables": "Non Current Receivables",
    "Non Current Related Party Assets": "Non Current Related Party Assets",
    "Other Fixed Assets": "Other Fixed Assets",
    "Goodwill": "Goodwill",
    "Related Party Assets - Intangible": "Related Party Assets - Intangible",
    "Other Intangible Assets": "Other Intangible Assets",
    "Short-term Debt": "Overdraft and Short Term Debt",
    "Current Maturities of Long-term Debt": "Current Maturities, Long Term Debt",
    "Subordinated Debt": "Subordinated Debt",
    "Accounts Payable": "Accts Payable - Trade (Trade Creditors)",
    "Other Payables": "Other Payables (Other Creditors)",
    "Income Taxes Payable": "Income Taxes Payable",
    "Other Taxation and Social Security": "Other Taxation and Social Security",
    "Current Related Party Liabilities": "Current Related Party Liabilities",
    "Accrued Expenses": "Accrued expenses and prepaid income",
    "Other Current Liabilities": "Other Liabilities - Current",
    "Long-term Debt": "Long Term Debt",
    "Long-term Debt - Subordinated": "Long Term Debt - Subordinated",
    "Non-Current Related Party Liabilities": "Related Party Liabilities - Non Current",
    "Provisions and Deferred Taxes": "Provisions and Deferred Taxes",
    "Other Non-Current Liabilities": "Other Non-Current Liabilities",
    "Share Capital": "Share Capital / Paid In Capital",
    "Retained Earnings": "Retained Earnings",
    "Other Equity": "Other Equity",
    "Translation Adjustment": "Translation Adjustment",
    "Minority Interests": "Minority Interests",
    "Current Assets": "CURRENT ASSETS",
    "Non-Current Assets": "NON CURRENT ASSETS",
    "Total Assets": "TOTAL ASSETS",
    "Current Liabilities": "CURRENT LIABILITIES",
    "Non-Current Liabilities": "NON CURRENT LIABILITIES",
    "Total Liabilities": "TOTAL LIABILITIES",
    "Equity": "EQUITY",
    "Total Equity & Liabilities": "TOTAL EQUITY & LIABILITIES"
}

# Function to extract tables from raw text
def extract_table(section_title, raw_text):
    lines = raw_text.split("\n")
    table_data = []

    for line in lines:
        columns = line.split()
        if len(columns) >= 3:
            item = " ".join(columns[:-2])
            year_1 = columns[-2]
            year_2 = columns[-1]

            # Standardize names using vocabulary mapping
            item = vocabulary_mapping.get(item, item)

            table_data.append([item, year_1, year_2])

    # Convert to DataFrame
    df = pd.DataFrame(table_data, columns=["Line Item", "Year 1", "Year 2"])

    # Convert to Markdown Table Format
    markdown_table = tabulate(df, headers="keys", tablefmt="github", showindex=False)

    return f"### {section_title}\n\n{markdown_table}\n"

# Extract relevant sections based on keywords
table_keywords = {
    "STATEMENT OF FINANCIAL POSITION": "Balance Sheet",
    "INCOME STATEMENT": "Income Statement",
    "STATEMENT OF COMPREHENSIVE INCOME": "Cash Flow Statement",
}

extracted_sections = {}
for keyword, title in table_keywords.items():
    if keyword in full_text:
        section_start = full_text.find(keyword)
        section_end = full_text.find("\n\n", section_start)
        extracted_sections[title] = extract_table(title, full_text[section_start:section_end])

# Combine extracted tables in Markdown
markdown_output = "\n\n".join(extracted_sections.values())

# Save output to Markdown file
with open("financial_report.md", "w") as md_file:
    md_file.write(markdown_output)

# Print Markdown Output
print(markdown_output)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
### Balance Sheet

| Line Item                                                                                          | Year 1    | Year 2    |
|----------------------------------------------------------------------------------------------------|-----------|-----------|
| STATEMENT OF                                                                                       | FINANCIAL | POSITION  |
| AS AT 31                                                                                           | MARCH     | 2020      |
| Company                                                                                            | No:       | 01471587  |
| Note                                                  

In [36]:
# The balance_sheet was converted from png to jpg for relabelling the names of the rows in this step

import os
import cv2
import pytesseract
import numpy as np
import pandas as pd
from pdf2image import convert_from_path
from markdownify import markdownify as md
from tabulate import tabulate  # For Markdown table formatting

# Define PDF path
PDF_PATH = "/content/vodafone_annual_report_reduced.pdf"

# Convert PDF pages to images
images = convert_from_path(PDF_PATH, dpi=300)

# Function to preprocess image for OCR
def preprocess_image(image):
    gray = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Binarization
    return binary

# Extract tables using OCR
def extract_text_from_image(image):
    processed_img = preprocess_image(image)
    extracted_text = pytesseract.image_to_string(processed_img, config="--psm 6")  # OCR
    return extracted_text

# Extract text from all images
extracted_text = [extract_text_from_image(img) for img in images]
full_text = "\n".join(extracted_text)

# Save extracted text for debugging
with open("extracted_text.txt", "w") as f:
    f.write(full_text)

# Vocabulary List for Standardization
vocabulary_mapping = {
    "Cash and Equivalents": "Cash",
    "Trade Receivables": "Accts Rec-Trade (Trade Debtors)",
    "Inventory": "Inventory (Stock)",
    "Tax Receivable": "Tax Receivable",
    "Other Current Assets": "Other Current Assets / Other Debtors",
    "Current Related Party Assets": "Current Related Party Assets",
    "Prepayments and Accrued Income": "Prepayments and accrued income",
    "Land & Buildings": "Land & Buildings",
    "Plant & Equipment": "Plant & Machinery",
    "Non Current Receivables": "Non Current Receivables",
    "Non Current Related Party Assets": "Non Current Related Party Assets",
    "Other Fixed Assets": "Other Fixed Assets",
    "Goodwill": "Goodwill",
    "Related Party Assets - Intangible": "Related Party Assets - Intangible",
    "Other Intangible Assets": "Other Intangible Assets",
    "Short-term Debt": "Overdraft and Short Term Debt",
    "Current Maturities of Long-term Debt": "Current Maturities, Long Term Debt",
    "Subordinated Debt": "Subordinated Debt",
    "Accounts Payable": "Accts Payable - Trade (Trade Creditors)",
    "Other Payables": "Other Payables (Other Creditors)",
    "Income Taxes Payable": "Income Taxes Payable",
    "Other Taxation and Social Security": "Other Taxation and Social Security",
    "Current Related Party Liabilities": "Current Related Party Liabilities",
    "Accrued Expenses": "Accrued expenses and prepaid income",
    "Other Current Liabilities": "Other Liabilities - Current",
    "Long-term Debt": "Long Term Debt",
    "Long-term Debt - Subordinated": "Long Term Debt - Subordinated",
    "Non-Current Related Party Liabilities": "Related Party Liabilities - Non Current",
    "Provisions and Deferred Taxes": "Provisions and Deferred Taxes",
    "Other Non-Current Liabilities": "Other Non-Current Liabilities",
    "Share Capital": "Share Capital / Paid In Capital",
    "Retained Earnings": "Retained Earnings",
    "Other Equity": "Other Equity",
    "Translation Adjustment": "Translation Adjustment",
    "Minority Interests": "Minority Interests",
    "Current Assets": "CURRENT ASSETS",
    "Non-Current Assets": "NON CURRENT ASSETS",
    "Total Assets": "TOTAL ASSETS",
    "Current Liabilities": "CURRENT LIABILITIES",
    "Non-Current Liabilities": "NON CURRENT LIABILITIES",
    "Total Liabilities": "TOTAL LIABILITIES",
    "Equity": "EQUITY",
    "Total Equity & Liabilities": "TOTAL EQUITY & LIABILITIES"
}

# Function to extract tables from raw text
def extract_table(section_title, raw_text):
    lines = raw_text.split("\n")
    table_data = []

    for line in lines:
        columns = line.split()
        if len(columns) >= 3:
            item = " ".join(columns[:-2])
            year_1 = columns[-2]
            year_2 = columns[-1]

            # Standardize names using vocabulary mapping
            item = vocabulary_mapping.get(item, item)

            table_data.append([item, year_1, year_2])

    # Convert to DataFrame
    df = pd.DataFrame(table_data, columns=["Line Item", "Year 1", "Year 2"])

    # Convert to Markdown Table Format
    markdown_table = tabulate(df, headers="keys", tablefmt="github", showindex=False)

    return df, f"### {section_title}\n\n{markdown_table}\n"

# Extract relevant sections based on keywords
table_keywords = {
    "STATEMENT OF FINANCIAL POSITION": "Balance Sheet",
}

extracted_sections = {}
csv_filename = "balance_sheet.csv"  # CSV file to store Balance Sheet data
for keyword, title in table_keywords.items():
    if keyword in full_text:
        section_start = full_text.find(keyword)
        section_end = full_text.find("\n\n", section_start)
        df, markdown_table = extract_table(title, full_text[section_start:section_end])
        extracted_sections[title] = markdown_table

        # Save only the Balance Sheet as CSV
        if title == "Balance Sheet":
            df.to_csv(csv_filename, index=False)
            print(f"Balance Sheet saved as {csv_filename}")

# Combine extracted tables in Markdown
markdown_output = "\n\n".join(extracted_sections.values())

# Save output to Markdown file
with open("financial_report.md", "w") as md_file:
    md_file.write(markdown_output)

# Print Markdown Output
print(markdown_output)


Balance Sheet saved as balance_sheet.csv
### Balance Sheet

| Line Item                                                                                          | Year 1    | Year 2    |
|----------------------------------------------------------------------------------------------------|-----------|-----------|
| STATEMENT OF                                                                                       | FINANCIAL | POSITION  |
| AS AT 31                                                                                           | MARCH     | 2020      |
| Company                                                                                            | No:       | 01471587  |
| Note                                                                                               | 2020      | 2019      |
| £m                                                                                                 | '         | €m        |
| Intangible assets 8                              

In [35]:
#Question 2 : Re-label the extracted Balance Sheet table line items using the closer corresponding terms from the attached vocabulary list
'''The balance sheet, in the form of CSV, is uploaded, and the old terms are being replaced by the allowed terms using the replace
   function '''


import pandas as pd
from tabulate import tabulate

# Load balance sheet data
df = pd.read_csv("balance_sheet.csv")

# Mapping of old terms to allowed terms
allowed_terms = {
    "Cash and cash equivalents": "Cash",
    "Trade and other receivables": "Accts Rec-Trade (Trade Debtors)",
    "Inventories": "Inventory (Stock)",
    "Intangible assets": "Other Intangible Assets",
    "Property, plant and equipment": "Land & Buildings",
    "Post-employment benefits - asset": "Related Party Assets - Intangible",
    "Creditors: amounts falling due within one year": "Accts Payable - Trade (Trade Creditors)",
    "Creditors: amounts falling due after more than one year": "Other Liabilities - Current",
    "Provisions for liabilities": "Provisions and Deferred Taxes",
    "Post-employment benefits - liability": "Related Party Liabilities - Non Current",
    "Called up share capital": "Share Capital / Paid In Capital",
    "Retained earnings": "Retained Earnings"
}

# Apply vocabulary changes to the "Line Item" column
df["Line Item"] = df["Line Item"].replace(allowed_terms, regex=True)

# Save the updated DataFrame to a new CSV file (optional)
df.to_csv("updated_balance_sheet.csv", index=False)

# Convert DataFrame to Markdown format with heading
markdown_output = "## Re-labelled Balance Sheet\n\n" + tabulate(df, headers="keys", tablefmt="github", showindex=False)

# Print Markdown output
print(markdown_output)

# Save Markdown table to a file (optional)
with open("updated_balance_sheet.md", "w") as md_file:
    md_file.write(markdown_output)


## Re-labelled Balance Sheet

| Line Item                                                                                          | Year 1    | Year 2    |
|----------------------------------------------------------------------------------------------------|-----------|-----------|
| STATEMENT OF                                                                                       | FINANCIAL | POSITION  |
| AS AT 31                                                                                           | MARCH     | 2020      |
| Company                                                                                            | No:       | 01471587  |
| Note                                                                                               | 2020      | 2019      |
| £m                                                                                                 | '         | €m        |
| Other Intangible Assets 8                                                      